# Runs 

<div style="text-align:center">
    <img src="../../images/mlflow_run.jpeg" alt="MLFlow Run" />
</div>


A MLflow run is a unit of work in MLflow that represents the execution of a machine learning experiment or a piece of code. It tracks the parameters, metrics, artifacts, and metadata associated with the run. MLflow runs allow you to log and track experiments, compare different runs, and reproduce results. Each run is associated with an experiment and can have multiple tags, parameters, metrics, and artifacts.

## Creating a MLflow Run

### Using start_run

```python
mlflow.start_run()
```

In [1]:
import mlflow 
mlflow.login()

2024/05/26 09:00:54 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://adb-3088650010345545.5.azuredatabricks.net.


In [2]:
experiment_name = "/Shared/Experiments/creating_runs"
tags = {"project_name":"UNDEFINED", "topic":"run_management"}
experiment_id = mlflow.create_experiment(name=experiment_name, tags=tags)
experiment = mlflow.set_experiment(experiment_name)

In [3]:
run_tags = {"tag1": "value1", "tag2": "value2"}
run = mlflow.start_run(
    run_name = "Run 1",
    experiment_id = experiment.experiment_id,
    description="This is a run description",
    tags= run_tags
)

In [5]:
run.to_dictionary()

{'info': {'artifact_uri': 'dbfs:/databricks/mlflow-tracking/2354118570535035/1be3f35ef3054b59bada6e9c31d7b229/artifacts',
  'end_time': None,
  'experiment_id': '2354118570535035',
  'lifecycle_stage': 'active',
  'run_id': '1be3f35ef3054b59bada6e9c31d7b229',
  'run_name': 'Run 1',
  'run_uuid': '1be3f35ef3054b59bada6e9c31d7b229',
  'start_time': 1716732173200,
  'status': 'RUNNING',
  'user_id': ''},
 'data': {'metrics': {},
  'params': {},
  'tags': {'mlflow.note.content': 'This is a run description',
   'mlflow.runName': 'Run 1',
   'mlflow.source.name': 'c:\\Users\\manue\\projects\\mlflow_for_ml_dev\\.venv\\Lib\\site-packages\\ipykernel_launcher.py',
   'mlflow.source.type': 'LOCAL',
   'mlflow.user': '8443236376042681',
   'tag1': 'value1',
   'tag2': 'value2'}}}

In [4]:
active_run = mlflow.active_run()
print(type(active_run))

<class 'mlflow.tracking.fluent.ActiveRun'>


In [6]:
mlflow.end_run()

#### Using the context Manager


> About `mlflow.start_run()`
> 
> The return value of `mlflow.start_run()` can be used as a context manager within a `with` block. Otherwise, you must call `end_run()` to terminate the current run.

Example:

```python
with mlflow.start_run() as run:
    print("Log metrics and params")
```

In [7]:
mlflow.end_run() # end the active run to start a new one
with mlflow.start_run(run_name="Run 2", experiment_id=experiment.experiment_id) as run:
    active_run = mlflow.active_run()
    print(type(active_run))  
    print("Active Run: ", run.info.run_id)
    print("\n \n")


# outside the with block
active_run = mlflow.active_run()
print(type(active_run))    

<class 'mlflow.tracking.fluent.ActiveRun'>
Active Run:  015f963f3bef4bf990a669c58f2b8488

 

<class 'NoneType'>


## Using MlflowClient

In [8]:
client = mlflow.MlflowClient()

In [9]:
created_run = client.create_run(experiment_id=experiment.experiment_id, run_name="test_run", tags={"tag1": "value1", "tag2": "value2"})

In [10]:
run = mlflow.active_run()
type(run)

NoneType

In [11]:
# Loging to MLflow without an active run will create a new run
mlflow.log_param("param1", 5)
run = mlflow.active_run()
type(run)

5

In [14]:
print(run.info)

<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/2354118570535035/fdb2eee6a99943a88d54b0d62a28a32e/artifacts', end_time=None, experiment_id='2354118570535035', lifecycle_stage='active', run_id='fdb2eee6a99943a88d54b0d62a28a32e', run_name='popular-quail-822', run_uuid='fdb2eee6a99943a88d54b0d62a28a32e', start_time=1716732278564, status='RUNNING', user_id=''>


## Updating Run Tags

In [16]:
# end the active run to start a new one
mlflow.end_run()
with mlflow.start_run(run_name="Run 3", experiment_id=experiment.experiment_id) as run:
    # set a single tag
    mlflow.set_tag("tag3", "value3")
    
    # Set multiple tags
    mlflow.set_tags({"tag4": "value4", "tag5": "value5"})

## Update run description

In [17]:
with mlflow.start_run(run_name="Run 4", experiment_id=experiment.experiment_id) as run:
    #Update description
    mlflow.set_tag("mlflow.note.content", "This is a new description")

In [18]:
# including markdown
with mlflow.start_run(run_name="Run 5", experiment_id=experiment.experiment_id) as run:
    #Update description
    mlflow.set_tag("mlflow.note.content", "# This is a new description")

## Retrieve run information

In [19]:
run = mlflow.get_run(run_id = run.info.run_id)

In [21]:
run.info

<RunInfo: artifact_uri='dbfs:/databricks/mlflow-tracking/2354118570535035/9ab046ba40314249a51b5fd2e84e4969/artifacts', end_time=1716732557480, experiment_id='2354118570535035', lifecycle_stage='active', run_id='9ab046ba40314249a51b5fd2e84e4969', run_name='Run 5', run_uuid='9ab046ba40314249a51b5fd2e84e4969', start_time=1716732556822, status='FINISHED', user_id=''>

In [22]:
run.data

<RunData: metrics={}, params={}, tags={'mlflow.note.content': '# This is a new description',
 'mlflow.runName': 'Run 5',
 'mlflow.source.name': 'c:\\Users\\manue\\projects\\mlflow_for_ml_dev\\.venv\\Lib\\site-packages\\ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'manuel.gil-m@outlook.com'}>

In [28]:
run.to_proto()

info {
  run_uuid: "9ab046ba40314249a51b5fd2e84e4969"
  experiment_id: "2354118570535035"
  run_name: "Run 5"
  user_id: ""
  status: FINISHED
  start_time: 1716732556822
  end_time: 1716732557480
  artifact_uri: "dbfs:/databricks/mlflow-tracking/2354118570535035/9ab046ba40314249a51b5fd2e84e4969/artifacts"
  lifecycle_stage: "active"
  run_id: "9ab046ba40314249a51b5fd2e84e4969"
}
data {
  tags {
    key: "mlflow.note.content"
    value: "# This is a new description"
  }
  tags {
    key: "mlflow.runName"
    value: "Run 5"
  }
  tags {
    key: "mlflow.source.name"
    value: "c:\\Users\\manue\\projects\\mlflow_for_ml_dev\\.venv\\Lib\\site-packages\\ipykernel_launcher.py"
  }
  tags {
    key: "mlflow.source.type"
    value: "LOCAL"
  }
  tags {
    key: "mlflow.user"
    value: "manuel.gil-m@outlook.com"
  }
}
inputs {
}